<a href="https://colab.research.google.com/github/ksawczak/dw_matrix/blob/master/day5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.5MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook

In [5]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [0]:
df = pd.read_csv('data/shoes_prices.csv', low_memory=False)
df_usd = df[df.prices_currency == 'USD'].copy()
df_usd['prices_amountmin'] = df_usd.prices_amountmin.astype(np.float)
filter_max = np.percentile(df_usd['prices_amountmin'],99)
df_usd_filter = df_usd[df_usd['prices_amountmin']<filter_max]

In [11]:
df = df_usd_filter
df.shape

(18280, 48)

In [0]:
df.to_csv('data/shoes_prices.csv', index=False)

In [0]:
def run_model (feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[feats].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring='neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

In [18]:
df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


(-58.133398968282776, 4.206122611474276)

In [20]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'],model)

(-57.31783843165656, 4.181246596160967)

In [21]:
df.features

0        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
1        [{"key":"Gender","value":["Men"]},{"key":"Shoe...
2        [{"key":"Gender","value":["Men"]},{"key":"Colo...
3        [{"key":"Gender","value":["Men"]},{"key":"Colo...
4        [{"key":"Gender","value":["Men"]},{"key":"Colo...
                               ...                        
19382    [{"key":"Style","value":["Desert Boots"]},{"ke...
19383    [{"key":"Shoe Size","value":["9.5"]},{"key":"G...
19384    [{"key":"Gender","value":["Unisex"]},{"key":"F...
19385    [{"key":"Gender","value":["Unisex"]},{"key":"F...
19386    [{"key":"Gender","value":["Unisex"]},{"key":"F...
Name: features, Length: 18280, dtype: object

Start here

In [33]:
def parse_features(x):
  output_dict = {}
  if str(x) == 'nan': return output_dict


  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item ['value'][0].lower().strip()

    output_dict[key] = value

    return output_dict

df ['features_parsed'] = df['features'].map(parse_features)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


In [34]:
df['features_parsed'].head().values

array([{'gender': 'men'}, {'gender': 'men'}, {'gender': 'men'},
       {'gender': 'men'}, {'gender': 'men'}], dtype=object)